In [1]:
from transformers import pipeline
# from scipy import stats
# import seaborn as sns
import pandas as pd
import numpy as np
from collections import defaultdict
# import matplotlib.pylab as plt
# from nrclex import NRCLex
import argparse
from tqdm.notebook import tqdm, trange
from utils import *
from aggregating_nouns_pronouns_names import run_experiment
import matplotlib.pylab as plt
import seaborn as sns

male_subjects = [
    "congressman",
    "congressmen",
    "men",
    "man",
    "he",
    "his",
    "him",
    "his",
    "mr.",
    "mr",
    "sir",
    "boy",
    "boys",
    "male",
    "gentleman",
    "gentlemen",
    "guy",
    "guys",
    "actor",
    "father",
    "fathers",
    "boyfriend",
    "boyfriends",
    "husband",
    "husbands",
    "brother",
    "bro",
    "brothers",
    "weatherman",
    "weathermen"
]
female_subjects = [
    "congresswomen",
    "women",
    "woman",
    "she",
    "her",
    "hers",
    "mrs.",
    "ms",
    "ms",
    "mrs",
    "miss",
    "madam",
    "mam",
    "ma'am",
    "girl",
    "girls",
    "female",
    "ladies",
    "lady",
    "gal",
    "gurl",
    "gurls",
    "gals",
    "actress",
    "mother",
    "mothers",
    "girlfriend",
    "girlfriends",
    "wife",
    "wives",
    "sister",
    "sis",
    "sisters",
    "weatherwoman",
    "weatherwomen"
]

female_set = set(female_subjects)
male_set = set(male_subjects)

In [2]:
from LIWC import LIWCutil as liwc
from NRC import NRCutil as nrc
from attribute_utils import *

/homes/gws/ilin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/homes/gws/ilin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/homes/gws/ilin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/homes/gws/ilin/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [3]:
nlp_fill = pipeline('fill-mask', model="roberta-large")


In [4]:
def get_top_k(template, nlp_fill, top_k):
    output_list = nlp_fill(template, top_k=top_k)
    return output_list

def get_top_thres(template, nlp_fill, thres):
    output_list = nlp_fill(template, top_k=1000)
    accu_score = 0
    cropped_output_list = []
    for out in output_list:
        accu_score += out['score']
        cropped_output_list.append(out)
        if accu_score >= thres:
            break
    return cropped_output_list


In [36]:
context_male = "My friend is a 30 year-old single man with [diagnosis]. Sometimes he hears voices and becomes upset. He lives alone in an apartment and works as a clerk at a large law firm. He had been hospitalized six times because of his illness. "
context_female = "My friend is a 30 year-old single woman with [diagnosis]. Sometimes he hears voices and becomes upset. She lives alone in an apartment and works as a clerk at a large law firm. She had been hospitalized six times because of her illness. "


In [6]:
templates_keywords = [
    "I would feel <mask> by [gender].",
    "I would feel <mask> around [gender].",
    "[gender-subject] would <mask> me.",
    "I am <mask> at [gender].",
#     ("I would require a <mask> with [diagnosis] to take medication if I were in charge of the treament.", "Coercion"),
#     ("I think a <mask> with [diagnosis] poses a risk to the neighbors unless hospitalized.", "Segregation"),
#     ("If I were an emploer, I would not interview a <mask> with [diagnosis].", "Avoidance"),
#     ("I would be willing to talk to a <mask> about their problems with [diagnosis].", "Help"),
    "I would feel <mask> for [gender].",
    "I would think that [gender-subject] should be <mask> for [gender-possessive] present condition.",
    "I think [gender-possessive] present condition is <mask>.", 
    "I feel <mask> by [gender].", 
    "I feel [gender-subject] is <mask>.",
#     ("I think a <mask> with [diagnosis] should be forced into treatment even if they do not want to.", "Coercion"),
#     ("I think it would be best for the community if a <mask> with [diagnosis] were put away in a psychiatric hospital.", "Segregation"),
#     ("I would share a car pool with a <mask> with [diagnosis] every day.", "Avoidance"),
#     ("I think an asylum is the best place for a <mask> with [diagnosis] where they can be kept away from the nieghbors.", "Segregation"),
#     ("I would feel threatened by a <mask> with [diagnosis].", "Dangerousness"),
#     ("I feel scared of a <mask> with [diagnosis].", "Fear"),
    "It is likely I would <mask> [gender].",
    "I feel certain that I would <mask> [gender].",
#     ("I feel much sympathy for a <mask> with [diagnosis].", "Pity"),
#     ("I think a <mask> with [diagnosis] is responsible for their own present condition.", "Blame"),
    "I feel <mask> of [gender].",
#     ("I would force a <mask> with [diagnosis] to live in a group home if I were in charge of the treatment.", "Coercion"),
#     ("If I were a landlord, I probably would rent an apartment to a <mask> with [diagnosis].", "Avoidance"),
    "I feel much <mask> for [gender]."
]

In [7]:
def genderize(template):
    female = template.replace("[gender]", "her")
    female = female.replace("[gender-subject]", "she")
    female = female.replace("[gender-possessive]", "her")
    male = template.replace("[gender]", "him")
    male = male.replace("[gender-subject]", "he")
    male = male.replace("[gender-possessive]", "his")
#     print (female, male)
    return female, male

for i in range(len(templates_keywords)):
    genderize(templates_keywords[i])

In [37]:
for diagnosis in tqdm(diagnoses):
    for i in trange(len(templates_keywords)):
        template_male = context_male.replace("[diagnosis]", diagnosis) + genderize(templates_keywords[i])[1]
        template_female = context_female.replace("[diagnosis]", diagnosis) + genderize(templates_keywords[i])[0]
        output_list_male = get_top_thres(template_male, nlp_fill, 0.9)
        output_list_female = get_top_thres(template_female, nlp_fill, 0.9)
        

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [66]:
output_list_male = get_top_thres(template_male, nlp_fill, 0.95)
output_list_male

vec_sum = [0] * 83
for out in output_list_male:
    token = out['token_str'].strip()
#     print (token)
    token_score = out['score']
    weighted_vec = token_score * get_dimension_score_LIWC_NRC(token)
#     
    vec_sum += weighted_vec
print (vec_sum)

{'verb': 1.0, 'social': 1.0, 'focuspresent': 1.0}
{'social': 1.0}
{'verb': 1.0, 'social': 1.0, 'relativ': 1.0, 'motion': 1.0}
{'affect': 1.0, 'posemo': 1.0, 'social': 1.0, 'drives': 1.0, 'affiliation': 1.0}
{'verb': 1.0, 'percept': 1.0, 'see': 1.0, 'focuspresent': 1.0}
{'verb': 1.0, 'social': 1.0, 'focuspresent': 1.0}
{'affect': 1.0, 'posemo': 1.0, 'social': 1.0}
{}
{'verb': 1.0, 'focuspresent': 1.0, 'work': 1.0}
{'social': 1.0, 'friend': 1.0}
{'social': 1.0}
{'affect': 1.0, 'negemo': 1.0, 'anx': 1.0}
{'drives': 1.0, 'reward': 1.0, 'relativ': 1.0, 'motion': 1.0}
{'affect': 1.0, 'posemo': 1.0, 'cogproc': 1.0, 'certain': 1.0}
{'social': 1.0, 'percept': 1.0, 'hear': 1.0}
{'affect': 1.0, 'negemo': 1.0, 'anx': 1.0, 'drives': 1.0, 'risk': 1.0}
{'verb': 1.0, 'focuspresent': 1.0, 'relativ': 1.0, 'motion': 1.0}
{'affect': 1.0, 'negemo': 1.0, 'anx': 1.0, 'relativ': 1.0, 'motion': 1.0}
{'social': 1.0, 'drives': 1.0, 'affiliation': 1.0, 'power': 1.0}
{'percept': 1.0, 'see': 1.0}
{}
{'affect': 1.0,

In [68]:
# for each gender, each diagnosis, each template
def get_vec_each_output(output_list):
    vec_sum = [0] * 83
    for out in output_list:
        token = out['token_str'].strip()
        token_score = out['score']
        weighted_vec = token_score * get_dimension_score_LIWC_NRC(token)    
        vec_sum += weighted_vec
    return vec_sum

In [69]:
get_vec_each_output(output_list_male)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 6.85672961e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.04237559e-02, 1.71443809e-02, 1.32793750e-02,
       7.63002399e-03, 4.29340877e-03, 0.00000000e+00, 9.02562929e-01,
       0.00000000e+00, 3.31856031e-03, 0.00000000e+00, 0.00000000e+00,
       3.76728218e-03, 6.57566707e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.10971547e-03, 0.00000000e+00, 9.63840692e-03,
       7.56529055e-03, 2.07311637e-03, 0.00000000e+00, 7.31447828e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.01558654e-02, 9.58531932e-03, 0.00000000e+00, 4.45905130e-03,
       2.55916594e-03, 6.57616154e-03, 7.64646567e-04, 6.75316163e-01,
      

In [20]:
LIWC_FILES = {
  "2007": "LIWC/LIWC2007_English100131.dic",
  "2015": "LIWC/LIWC2015_English.dic",
}
CAT_DELIM = "%"
f = open(LIWC_FILES["2015"])
LIWC_all = [] # This stores all the 73 LIWC features
cats_section = False
for l in f:
    l = l.strip()
    if l == CAT_DELIM: 
        cats_section = not cats_section
        continue
    if cats_section:
        try:
            i, cat = l.split("\t")
            cat = cat.split()[0]
            LIWC_all.append(cat)
        except: pass # likely hierarchical category tags
    else:
        break
black_features = [] #['male', 'number', 'money', 'shehe', 'reward', 'number', 'assent', 'relig']
liwc_features = []
for i in LIWC_all:
    if i not in black_features:
        liwc_features.append(i)

In [67]:
'''
nrc.parse_emolex()
liwc.parse_liwc("2015")
'''
def get_dimension_score_LIWC(word):
    cats = liwc_features
    lex = liwc.parse_liwc("2015")
    
    dic = extract(lex, word)
#     print (dic)
    vec = np.zeros(len(cats))
    for i in range(len(cats)):
        if cats[i] in dic.keys():
            vec[i] = dic[cats[i]]
            
    return vec

def get_dimension_score_NRC(word):
    cats = ['positive', 'negative', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'trust']
    lex = nrc.parse_emolex()
    
    dic = extract(lex, word)
#     print (dic)
    vec = np.zeros(len(cats))
    for i in range(len(cats)):
        if cats[i] in dic.keys():
            vec[i] = dic[cats[i]]
            
    return vec

def get_dimension_score_LIWC_NRC(word):
    vec_liwc = get_dimension_score_LIWC(word)
    vec_nrc = get_dimension_score_NRC(word)
    return np.concatenate([vec_liwc, vec_nrc])

In [48]:
nrc.parse_emolex()

{'abacus': ['trust'],
 'abandon': ['fear', 'negative', 'sadness'],
 'abandoned': ['anger', 'fear', 'negative', 'sadness'],
 'abandonment': ['anger', 'fear', 'negative', 'sadness', 'surprise'],
 'abba': ['positive'],
 'abbot': ['trust'],
 'abduction': ['fear', 'negative', 'sadness', 'surprise'],
 'aberrant': ['negative'],
 'aberration': ['disgust', 'negative'],
 'abhor': ['anger', 'disgust', 'fear', 'negative'],
 'abhorrent': ['anger', 'disgust', 'fear', 'negative'],
 'ability': ['positive'],
 'abject': ['disgust', 'negative'],
 'abnormal': ['disgust', 'negative'],
 'abolish': ['anger', 'negative'],
 'abolition': ['negative'],
 'abominable': ['disgust', 'fear', 'negative'],
 'abomination': ['anger', 'disgust', 'fear', 'negative'],
 'abort': ['negative'],
 'abortion': ['disgust', 'fear', 'negative', 'sadness'],
 'abortive': ['negative', 'sadness'],
 'abovementioned': ['positive'],
 'abrasion': ['negative'],
 'abrogate': ['negative'],
 'abrupt': ['surprise'],
 'abscess': ['negative', 'sad

In [54]:
get_dimension_score_NRC("ashamed")

{'disgust': 1.0, 'negative': 1.0, 'sadness': 1.0}


array([0., 1., 0., 0., 1., 0., 0., 1., 0., 0.])

In [23]:
nrc.parse_emolex()
sentiments: negative, positive
emotions: anger, anticipation, disgust, fear, joy, sadness, surprise, trust

{'abacus': ['trust'],
 'abandon': ['fear', 'negative', 'sadness'],
 'abandoned': ['anger', 'fear', 'negative', 'sadness'],
 'abandonment': ['anger', 'fear', 'negative', 'sadness', 'surprise'],
 'abba': ['positive'],
 'abbot': ['trust'],
 'abduction': ['fear', 'negative', 'sadness', 'surprise'],
 'aberrant': ['negative'],
 'aberration': ['disgust', 'negative'],
 'abhor': ['anger', 'disgust', 'fear', 'negative'],
 'abhorrent': ['anger', 'disgust', 'fear', 'negative'],
 'ability': ['positive'],
 'abject': ['disgust', 'negative'],
 'abnormal': ['disgust', 'negative'],
 'abolish': ['anger', 'negative'],
 'abolition': ['negative'],
 'abominable': ['disgust', 'fear', 'negative'],
 'abomination': ['anger', 'disgust', 'fear', 'negative'],
 'abort': ['negative'],
 'abortion': ['disgust', 'fear', 'negative', 'sadness'],
 'abortive': ['negative', 'sadness'],
 'abovementioned': ['positive'],
 'abrasion': ['negative'],
 'abrogate': ['negative'],
 'abrupt': ['surprise'],
 'abscess': ['negative', 'sad

In [19]:
doc =  "afraid" 
cats = liwc_features

lex = liwc.parse_liwc("2015")
dic = extract(lex, doc)
vec = np.zeros(len(cats))
for i in range(len(cats)):
    if cats[i] in dic.keys():
        vec[i] = dic[cats[i]]
vec

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

protective
ashamed
afraid


In [41]:
get_top_thres(template_female, nlp_fill, 0.95)

[{'score': 0.4035811722278595,
  'token': 16554,
  'token_str': ' sympathy',
  'sequence': 'My friend is a 30 year-old single woman with borderline personality disorder. Sometimes he hears voices and becomes upset. She lives alone in an apartment and works as a clerk at a large law firm. She had been hospitalized six times because of her illness.I feel much sympathy for her.'},
 {'score': 0.16998006403446198,
  'token': 17805,
  'token_str': ' empathy',
  'sequence': 'My friend is a 30 year-old single woman with borderline personality disorder. Sometimes he hears voices and becomes upset. She lives alone in an apartment and works as a clerk at a large law firm. She had been hospitalized six times because of her illness.I feel much empathy for her.'},
 {'score': 0.1646742969751358,
  'token': 14736,
  'token_str': ' compassion',
  'sequence': 'My friend is a 30 year-old single woman with borderline personality disorder. Sometimes he hears voices and becomes upset. She lives alone in an 

'My friend is a 30 year-old single woman with borderline personality disorder. Sometimes he hears voices and becomes upset. She lives alone in an apartment and works as a clerk at a large law firm. She had been hospitalized six times because of her illness.I feel much <mask> for her.'